## **Data preparation**

In [32]:
import torch
from torchtext.legacy import data

mySeed = 1234

torch.manual_seed(mySeed)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)


In [33]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)


In [34]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(mySeed))


In [35]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')


Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [36]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)


In [37]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)


## **Model implementation**

In [38]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedded = self.dropout(self.embedding(text))

        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)

        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                            
        return self.fc(hidden)


In [39]:
input_dimensions = len(TEXT.vocab)
embed_dim = 100
hidden_dim = 256
output_dim = 1
layers = 2
bidir = True
dropout = 0.5
pad_idx = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(input_dimensions, 
            embed_dim, 
            hidden_dim, 
            output_dim, 
            layers, 
            bidir, 
            dropout, 
            pad_idx)


In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 4,810,857 trainable parameters


In [57]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)


torch.Size([25002, 100])


In [58]:
model.embedding.weight.data.copy_(pretrained_embeddings)


tensor([[-1.1172e-01, -4.9659e-01,  1.6307e-01,  ...,  1.2647e+00,
         -2.7527e-01, -1.3254e-01],
        [-8.5549e-01, -7.2081e-01,  1.3755e+00,  ...,  8.2522e-02,
         -1.1314e+00,  3.9972e-01],
        [-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -1.4590e-01,
          8.2780e-01,  2.7062e-01],
        ...,
        [-1.8866e-01,  7.5537e-01, -1.6210e-01,  ...,  7.8887e-05,
          2.2752e-01, -1.9435e-01],
        [ 8.1298e-02,  2.4855e-01,  4.3583e-01,  ..., -3.6564e-01,
          6.6258e-01,  5.0125e-01],
        [ 7.9125e-01, -2.5157e-03,  8.3469e-01,  ...,  2.5457e-01,
          8.3578e-01,  1.2827e+00]], device='cuda:0')

In [59]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embed_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embed_dim)

print(model.embedding.weight.data)


tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-3.8194e-02, -2.4487e-01,  7.2812e-01,  ..., -1.4590e-01,
          8.2780e-01,  2.7062e-01],
        ...,
        [-1.8866e-01,  7.5537e-01, -1.6210e-01,  ...,  7.8887e-05,
          2.2752e-01, -1.9435e-01],
        [ 8.1298e-02,  2.4855e-01,  4.3583e-01,  ..., -3.6564e-01,
          6.6258e-01,  5.0125e-01],
        [ 7.9125e-01, -2.5157e-03,  8.3469e-01,  ...,  2.5457e-01,
          8.3578e-01,  1.2827e+00]], device='cuda:0')


In [60]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())


In [61]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)


In [62]:
def binary_accuracy(preds, y):
  
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


## **Model Training**

In [63]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [64]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [65]:
N_EPOCHS = 4

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'myModel.pt')
    
    print('Epoch:')
    print(epoch+1)
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch:
1
	Train Loss: 0.456 | Train Acc: 78.73%
	 Val. Loss: 0.326 |  Val. Acc: 86.22%
Epoch:
2
	Train Loss: 0.337 | Train Acc: 85.48%
	 Val. Loss: 0.290 |  Val. Acc: 87.06%
Epoch:
3
	Train Loss: 0.295 | Train Acc: 87.90%
	 Val. Loss: 0.316 |  Val. Acc: 85.57%
Epoch:
4
	Train Loss: 0.246 | Train Acc: 90.10%
	 Val. Loss: 0.251 |  Val. Acc: 89.84%


In [66]:
model.load_state_dict(torch.load('myModel.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')


Test Loss: 0.265 | Test Acc: 89.52%
